In [ ]:
from datetime import datetime, timedelta, date
import numpy as np
from numpy.linalg import norm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import base64
import csv
import random
import shutil
import time
import glob
import altair as alt
from altair import expr, datum
from tqdm import tqdm
import pickle
import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms, datasets, models
from torchvision.transforms import Compose
from torch.nn import Linear
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score, accuracy_score

from scipy import signal
from scipy.ndimage import gaussian_filter

from itertools import combinations
from skimage import io
from skimage.metrics import structural_similarity as ssim

import rasterio
from rasterio.plot import show


## Preparing the data

Doing a basic EDA we realized that certain images were taken in the same places (they have the same lot/lan) but with different dates. These images, even though they were not identical, were quite similar, so they may bias our model.

Problems that can arise with these images:

- Data leakage (if we leave the images and proceed without treating them differently, we will have similar images in the training and validation set and the accuracy of our model in our validation set will be higher than it actually is).
A solution for this problem is to separate the images by "groups" of locations and make sure that each group belongs to either the validation or the test set. This way we are avoiding data leakage and making sure that our model is going to be validated with images that he never encounter before
     

     
- However, when following this approach we may end up overfitting. Since our model is being trained with really similar images. This will cause the model to classify very well the images of the places we have given it but when it encounters new places it will perform badly, i.e. overfitting. To avoid this problem, we must treat our data first. For example, for each group of images, we will detect which ones of them are really similar (using SSIM - Structural Similarity Index) and take out the ones with this index above a certain threshold. By doing this, we will obviously lose some of our data so later on to train our model, we will have to make sure that we create new samples with data augmentation.

## Classes and Functions

In [ ]:
# Model
class CNN(nn.Module):
    """
    Convolutional Neural Network (CNN) model for binary classification.

    This CNN model consists of convolutional layers followed by fully connected layers
    for binary classification.

    Attributes:
        conv1 (nn.Conv2d): First convolutional layer.
        pool1 (nn.MaxPool2d): First max-pooling layer.
        conv2 (nn.Conv2d): Second convolutional layer.
        pool2 (nn.MaxPool2d): Second max-pooling layer.
        conv3 (nn.Conv2d): Third convolutional layer.
        pool3 (nn.MaxPool2d): Third max-pooling layer.
        fc1 (nn.Linear): First fully connected layer.
        fc2 (nn.Linear): Second fully connected layer.
        sigmoid (nn.Sigmoid): Sigmoid activation function for binary classification.

    Methods:
        __init__(self): Initialize the CNN model layers.
        forward(self, x): Forward pass through the model.

    """
    def __init__(self):
        
        """
        Initialize the CNN model layers.

        The model includes convolutional layers, max-pooling layers, and fully connected
        layers for binary classification.
        """
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  # Accepts 1 input channel for grayscale images
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * (img_size[0] // 8) * (img_size[1] // 8), 128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        
        """
        Forward pass through the model.

        Args:
            x (torch.Tensor): Input data tensor.

        Returns:
            torch.Tensor: Output tensor representing binary classification probabilities.
        """
        
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.pool3(x)
        x = x.view(-1, 128 * (img_size[0] // 8) * (img_size[1] // 8))
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


In [ ]:
class CustomTIFDataset(Dataset):
    """
    A custom PyTorch dataset class for working with .tif image files.

    This class loads and processes .tif images from a specified directory and provides
    them as a PyTorch dataset for training and validation.

    Args:
        root (str): The root directory containing .tif image files.
        transform (callable, optional): A function/transform to apply to the images.

    Attributes:
        root (str): The root directory where image files are located.
        transform (callable, optional): The transform function for image preprocessing.
        file_list (list): List of .tif image file paths.
        data (list): List of image data.
        labels (list): List of image labels.

    Methods:
        __init__(self, root, transform=None): Initializes the dataset by loading image data.
        __len__(self): Returns the number of images in the dataset.
        __getitem__(self, idx): Retrieves an image and its label by index.
        """
    
    def __init__(self, root, transform=None):    
        """
        Initialize the CustomTIFDataset.

        Args:
            root (str): The root directory containing .tif image files.
            transform (callable, optional): A function/transform to apply to the images.
            """
        
        self.root = root
        self.transform = transform
        self.file_list = []

        # Recursively search for .tif files in subdirectories
        for dirpath, dirnames, filenames in os.walk(root):
            for fname in filenames:
                if fname.endswith('.tif'):
                    self.file_list.append(os.path.join(dirpath, fname))

        self.data = []
        self.labels = []

        for image_path in self.file_list:
            with rasterio.open(image_path) as img:
                image = img.read(1).astype(np.float32)  # Adjust the band index as needed

                if self.transform:
                    image = self.transform(image)

                label = 0 if "no_plume" in image_path else 1  # Assign labels based on the image path

                self.data.append(image)
                self.labels.append(label)

        # Print the number of images loaded
        print(f"Loaded {len(self.data)} images")

    def __len__(self):
        """
        Get the number of images in the dataset.

        Returns:
            int: The number of images in the dataset.
            """
        
        return len(self.data)

    def __getitem__(self, idx):  
        """
        Get an image and its label by index.

        Args:
            idx (int): The index of the image to retrieve.

        Returns:
            tuple: A tuple containing the image data and its label.
            """
            
        image = self.data[idx]
        label = self.labels[idx]
        return image, label

In [ ]:
# Define a custom transformation to normalize the tensor
class MinMaxNormalize(object):
    """
    A custom transformation to normalize a PyTorch tensor between 0 and 1.

    This transformation calculates the minimum and maximum values in the input tensor
    and normalizes the tensor so that its values fall within the range [0, 1].

    Usage:
    transform = MinMaxNormalize()
    normalized_tensor = transform(input_tensor)

    Args:
        None

    Methods:
        __call__(self, tensor): Normalize the input tensor.

    Returns:
        torch.Tensor: The normalized tensor with values between 0 and 1. 
        """
        
    def __call__(self, tensor):
        """
        Normalize the input tensor.

        Args:
            tensor (torch.Tensor): The input tensor to be normalized.

        Returns:
            torch.Tensor: The normalized tensor with values between 0 and 1.
            """
        min_val = tensor.min()
        max_val = tensor.max()
        normalized_tensor = (tensor - min_val) / (max_val - min_val)
        return normalized_tensor

In [ ]:
class CustomRotate(object):
    """
    A custom transformation to rotate an image by a specified angle.

    This transformation rotates an image by the specified angle in degrees.

    Usage:
    transform = CustomRotate(degrees)
    rotated_image = transform(input_image)

    Args:
        degrees (float): The angle in degrees by which to rotate the image.

    Methods:
        __call__(self, image): Rotate the input image.

    Returns:
        numpy.ndarray: The rotated image.
        """
        
    def __init__(self, degrees):
        """
        Initialize the CustomRotate transformation.

        Args:
            degrees (float): The angle in degrees by which to rotate the image.
            """
        self.degrees = degrees

    def __call__(self, image):
        """
        Rotate the input image.

        Args:
            image (numpy.ndarray): The input image to be rotated.

        Returns:
            numpy.ndarray: The rotated image.
            """
        
        image = Image.fromarray(image)
        image = image.rotate(self.degrees)
        
        return np.array(image)
    


In [ ]:
class CustomHorizontalFlip(object):
    """
    A custom transformation to horizontally flip an image with a given probability.

    This transformation horizontally flips an image with the specified probability.

    Args:
        p (float, optional): The probability of flipping the image (default is 0.5).

    Methods:
        __init__(self, p): Initialize the CustomHorizontalFlip transformation.
        __call__(self, image): Horizontally flip the input image with the specified probability.

    Returns:
        numpy.ndarray: The horizontally flipped or original image, depending on the probability.
        """
    def __init__(self, p=0.5):
        """
         Initialize the CustomHorizontalFlip transformation.

        Args:
            p (float, optional): The probability of flipping the image (default is 0.5).
        """
        
        self.p = p

    def __call__(self, image):
        
        """
        Horizontally flip the input image with the specified probability.

        Args:
            image (numpy.ndarray): The input image to be flipped.

        Returns:
            numpy.ndarray: The horizontally flipped or original image, depending on the probability.
            """
        
        if np.random.random() < self.p:
            return np.fliplr(image).copy()
        
        return image

In [ ]:
# Calculate SSIM
def calculate_ssim(img1, img2):
    
    """
    Calculate the Structural Similarity Index (SSIM) between two images.

    The SSIM is a measure of structural similarity between two images. It takes into
    account luminance, contrast, and structure. A higher SSIM value indicates greater
    similarity between the images.

    Parameters:
    - img1 (numpy.ndarray): The first image (as a NumPy array).
    - img2 (numpy.ndarray): The second image (as a NumPy array).

    Returns:
    - float: The SSIM value, ranging from -1 to 1, with 1 indicating identical images.
    """
    
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())
    mu1 = signal.fftconvolve(img1, window, mode='same')
    mu2 = signal.fftconvolve(img2, window, mode='same')
    mu1_sq = mu1 * mu1
    mu2_sq = mu2 * mu2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = signal.fftconvolve(img1 * img1, window, mode='same') - mu1_sq
    sigma2_sq = signal.fftconvolve(img2 * img2, window, mode='same') - mu2_sq
    sigma12 = signal.fftconvolve(img1 * img2, window, mode='same') - mu1_mu2
    ssim_num = (2 * mu1_mu2 + 0.01) * (2 * sigma12 + 0.03)
    ssim_den = (mu1_sq + mu2_sq + 0.01) * (sigma1_sq + sigma2_sq + 0.03)
    
    return np.mean(ssim_num / ssim_den)

In [ ]:
# Define a function to reverse the min-max normalization
def reverse_min_max_normalize(normalized_tensor, min_val, max_val):
    
    """
    Reverse the Min-Max normalization of a tensor.

    This function reverses the Min-Max normalization by scaling the normalized tensor
    back to the original range specified by min_val and max_val.

    Args:
        normalized_tensor (torch.Tensor): The normalized tensor to be reversed.
        min_val (float): The minimum value of the original range before normalization.
        max_val (float): The maximum value of the original range before normalization.

    Returns:
        torch.Tensor: The reversed tensor with values scaled back to the original range.
    """   
    
    reversed_tensor = (normalized_tensor * (max_val - min_val)) + min_val
    
    return reversed_tensor

In [ ]:
def train_and_evaluate_model(model, train_loader, valid_loader, criterion, optimizer, device, num_epochs, log_interval=1):
    
    """
    Train and evaluate a PyTorch model on a training and validation dataset.

    This function trains a model on a training dataset, evaluates its performance on a validation dataset,
    and returns the best model based on validation AUC score.

    Args:
        model (nn.Module): The PyTorch model to be trained and evaluated.
        train_loader (DataLoader): The DataLoader for the training dataset.
        valid_loader (DataLoader): The DataLoader for the validation dataset.
        criterion (nn.Module): The loss function used for training.
        optimizer (torch.optim.Optimizer): The optimizer used for training.
        device (str): The device to run the model on (e.g., 'cuda' or 'cpu').
        num_epochs (int): The number of training epochs.
        log_interval (int, optional): The interval for logging training progress (default is 1).

    Returns:
        nn.Module: The best model based on the highest validation AUC score.
        list: List of AUC scores for the training dataset at each epoch.
        list: List of AUC scores for the validation dataset at each epoch.
        list: List of training loss values at each epoch.
        list: List of validation loss values at each epoch.
    """
    
    train_auc_all = []
    valid_auc_all = []
    train_loss_all = []
    valid_loss_all = []

    best_valid_auc = 0.0
    best_model = None

    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_predictions = []
        train_targets = []

        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.float().unsqueeze(1))
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            predicted = outputs
            train_predictions.extend(predicted.detach().cpu().numpy().flatten())
            train_targets.extend(labels.detach().cpu().numpy().flatten())

        train_auc = roc_auc_score(train_targets, train_predictions)
        train_loss /= len(train_loader)
        train_auc_all.append(train_auc)
        train_loss_all.append(train_loss)

        model.eval()
        valid_loss = 0.0
        valid_predictions = []
        valid_targets = []

        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels.float().unsqueeze(1))
                valid_loss += loss.item()

                preds = outputs
                valid_predictions.extend(preds.detach().cpu().numpy().flatten())
                valid_targets.extend(labels.detach().cpu().numpy().flatten())

        valid_auc = roc_auc_score(valid_targets, valid_predictions)
        valid_loss /= len(valid_loader)
        valid_auc_all.append(valid_auc)
        valid_loss_all.append(valid_loss)

        if valid_auc > best_valid_auc:
            best_valid_auc = valid_auc
            best_model = model

        if (epoch + 1) % log_interval == 0:
            print(f'Epoch {epoch+1}/{num_epochs}: '
                  f'Train Loss: {train_loss:.4f}, Train AUC: {train_auc:.4f}, '
                  f'Validation Loss: {valid_loss:.4f}, Validation AUC: {valid_auc:.4f}')

    return best_model, train_auc_all, valid_auc_all, train_loss_all, valid_loss_all


## Train and Validation Sets

In [ ]:
# paths to fetch the data
path = 'hfactory_magic_folders/cleanr/train data/metadata.csv'
path_1 = 'hfactory_magic_folders/cleanr/train data/images/plume'
path_2 = 'hfactory_magic_folders/cleanr/train data/images/no_plume'

In [ ]:
# Data loading
df = pd.read_csv(path)

# Store how many images have the same latitude and longitude (to avoid overfitting)
paths = df.groupby(["lat","lon","plume"]).count()[['path']].sort_values("path", ascending = False)

# Initialize an empty column to store all images path
paths['lists'] = np.nan

temp = []

# Save all images paths that have the same lat and lon in one list
for i,row in enumerate(paths.iterrows()):  
  row = row[0]
  temp.append(list(df['path'][(df['lat'] == row[0]) & (df['lon'] == row[1])]))
    
paths['lists'] = temp

# We don't want anymore the lat, lon and plume to be an index so we are resetting the dataframe
paths = paths.reset_index()

In [ ]:
# Load your into a list

directory = "hfactory_magic_folders/cleanr/train data/"
extension = ".tif"

# Create a list to store the paths of images to keep
final_group_plume = []
final_group_no_plume = []

for i in range(len(paths)):
    image_paths = paths.iloc[i]["lists"] 
    plume_or_not = paths.iloc[i]["plume"]
    image_paths2 = [directory + image_path + extension for image_path in image_paths]
    images = [io.imread(path) for path in image_paths2]

    # Define the similarity threshold (adjust as needed)
    similarity_threshold = 0.7
    
    # Create a list to store the paths of images to keep of a particular location
    final_group_loc = []

    # Create a list to track images to keep (initialize with all indices)
    images_to_keep = list(range(len(images)))

    # Create a set to track images already marked for removal
    marked_for_removal = set()

    # Calculate SSIM scores and mark duplicates for removal
    for i in range(len(images)):
        if i not in marked_for_removal:
            for j in range(i + 1, len(images)):
                if j not in marked_for_removal:
                    ssim_score = calculate_ssim(images[i], images[j])
                    if ssim_score > similarity_threshold:
                        # Mark one of the images for removal (add its index to the list)
                        images_to_keep.remove(j)
                        marked_for_removal.add(j)

    # Add the paths of images to keep to the final_group_loc
    final_group_loc.extend([image_paths2[i] for i in images_to_keep])
    
    # Add the paths of images to keep to the final_group_plume or final_group_not_plume
    if plume_or_not == "yes":
        final_group_plume.extend([final_group_loc])
    else:
        final_group_no_plume.extend([final_group_loc])


Now that we have the selected images divided by locations and plume or not plume, lets create our validation and training set.

In [ ]:
# Define the percentage for selection (10%)
percentage = 20

# Randomly select 10% of locations with plume
plume_validation = random.sample(final_group_plume, k=int(0.01 * percentage * len(final_group_plume)))
plume_training = [loc for loc in final_group_plume if loc not in plume_validation]

# Randomly select 10% of locations with no plume
no_plume_validation = random.sample(final_group_no_plume, k=int(0.01 * percentage * len(final_group_no_plume)))
no_plume_training = [loc for loc in final_group_no_plume if loc not in no_plume_validation]


In [ ]:
# Paths and data_dir
val_dir = 'validation'
train_dir = 'train'
plume_dir = 'plume'
no_plume_dir = 'no_plume'

# List of subdirectories
subdirectories = [plume_dir, no_plume_dir]
directories = [val_dir, train_dir]

# Clean the validation and train directories
for directory in directories:
    for subdirectory in subdirectories:
        path = os.path.join(directory, subdirectory)
        if os.path.exists(path):
            shutil.rmtree(path)  # Clean the directory if it already exists
        os.makedirs(path, exist_ok=True)  # Create the directory if it doesn't exist

In [ ]:
# Copy files to validation directories
for loc in plume_validation:
    for filename in loc:
        shutil.copy2(filename, "./validation/plume")

for loc in no_plume_validation:
    for filename in loc:
        shutil.copy2(filename, "./validation/no_plume")

for loc in plume_training:
    for filename in loc:
        shutil.copy2(filename, "./train/plume")

for loc in no_plume_training:
    for filename in loc:
        shutil.copy2(filename, "./train/no_plume")

Let's check our data.

In [ ]:
# Specify the path to your folder containing the images
folder_path1 = './train/no_plume'
folder_path2 = './train/plume'
folder_path3 = './validation/no_plume'
folder_path4 = './validation/plume'

# Define the file extensions that you consider as images (e.g., jpg, png, etc.)
image_extensions = ['tif']  # Add more extensions if needed

# Use glob to find all files in the folder with the specified extensions
image_files1 = []
image_files2 = []
image_files3 = []
image_files4 = []
for ext in image_extensions:
    image_files1.extend(glob.glob(os.path.join(folder_path1, f'*.{ext}')))
    image_files2.extend(glob.glob(os.path.join(folder_path2, f'*.{ext}')))
    image_files3.extend(glob.glob(os.path.join(folder_path3, f'*.{ext}')))
    image_files4.extend(glob.glob(os.path.join(folder_path4, f'*.{ext}')))

# Count the number of image files
num_images1 = len(image_files1)
num_images2 = len(image_files2)
num_images3 = len(image_files3)
num_images4 = len(image_files4)

print(f'Total number of images in the folder train no plume: {num_images1}')
print(f'Total number of images in the folder train plume: {num_images2}')
print(f'Total number of images in the folder validation no plume: {num_images3}')
print(f'Total number of images in the folder validation plume: {num_images4}')

In [ ]:
# Set the paths to the training and validation data
train_data_dir = './train'
valid_data_dir = './validation'

# Set the image size, batch size, and number of classes (1 for binary classification)
img_size = (64, 64)
batch_size = 32
num_classes = 1  # Binary classification

# Define data transformations
train_augmentations = Compose([
    CustomRotate(degrees=15),
    CustomHorizontalFlip(),
    transforms.ToTensor(),
    MinMaxNormalize(),
])

train_original_transform = transforms.Compose([
    #transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor(),
    MinMaxNormalize(),
])

valid_transform = transforms.Compose([
    #transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor(),
    MinMaxNormalize(),
])

# Create datasets and data loaders
# Load original data
train_original_dataset = CustomTIFDataset(train_data_dir, transform=train_original_transform)
train_augmented_dataset = CustomTIFDataset(train_data_dir, transform=train_augmentations)
train_dataset = ConcatDataset([train_original_dataset, train_augmented_dataset])
valid_dataset = CustomTIFDataset(valid_data_dir, transform=valid_transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# Example tensor (image, label)
example_tensor, label = valid_dataset[0]

# Reverse min-max normalization (assuming min_val and max_val are known)
min_val = 0.0
max_val = 1.0
reversed_tensor = reverse_min_max_normalize(example_tensor, min_val, max_val)

# Convert the reversed tensor to a NumPy array
reversed_image = reversed_tensor.numpy()

# Visualize the image
plt.imshow(reversed_image[0], cmap="gray")
plt.show()


In [ ]:
# Count "plume" and "no plume" instances in the training dataset
plume_count_train = len([item for item in train_dataset if item[1] == 0])
no_plume_count_train = len([item for item in train_dataset if item[1] == 1])

# Count "plume" and "no plume" instances in the validation dataset
plume_count_valid = len([item for item in valid_dataset if item[1] == 0])
no_plume_count_valid = len([item for item in valid_dataset if item[1] == 1])

print("Training Data - Plume: {} instances, No Plume: {} instances".format(plume_count_train, no_plume_count_train))
print("Validation Data - Plume: {} instances, No Plume: {} instances".format(plume_count_valid, no_plume_count_valid))


## Model 1: CNN

In [ ]:
model = CNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 30

In [ ]:
# Usage:
best_model, train_auc, valid_auc, train_loss, valid_loss = train_and_evaluate_model(
    model, train_loader, valid_loader, criterion, optimizer, device, num_epochs, log_interval=1)

# Plot the training and validation AUC and loss
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(train_auc, label='Train AUC')
plt.plot(valid_auc, label='Validation AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()
plt.title('Training and Validation AUC')

plt.subplot(1, 2, 2)
plt.plot(train_loss, label='Train Loss')
plt.plot(valid_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.show()

## Model 2: ResNet50

In [ ]:
# Load a pre-trained ResNet model (e.g., ResNet-50)
model = models.resnet50(pretrained=True)

# Modify the first convolutional layer to accept 1 channel grayscale images
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Freeze all layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match your problem
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 128),
    nn.ReLU(),
    nn.Linear(128, 1),  # Change the output size for binary classification (1 output neuron)
    nn.Sigmoid()  # Use sigmoid activation for binary classification
)

optimizer = optim.Adam(model.fc.parameters())  # Only optimize the final fully connected layer


In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 30

In [ ]:
# Usage:
best_model, train_auc, valid_auc, train_loss, valid_loss = train_and_evaluate_model(
    model, train_loader, valid_loader, criterion, optimizer, device, num_epochs, log_interval=1)

# Plot the training and validation AUC and loss
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(train_auc, label='Train AUC')
plt.plot(valid_auc, label='Validation AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()
plt.title('Training and Validation AUC')

plt.subplot(1, 2, 2)
plt.plot(train_loss, label='Train Loss')
plt.plot(valid_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.show()


## Model 3: Wide ResNet 50

In [ ]:
# Load a pre-trained Wide ResNet-50 model
model = models.wide_resnet50_2(pretrained=True)

# Modify the first convolutional layer to accept 1 channel grayscale images
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modify the final fully connected layer to match your problem
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 128),
    nn.ReLU(),
    nn.Linear(128, 1),  # Change the output size for binary classification (1 output neuron)
    nn.Sigmoid()  # Use sigmoid activation for binary classification
)

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 50

In [ ]:
# Usage:
best_model, train_auc, valid_auc, train_loss, valid_loss = train_and_evaluate_model(
    model, train_loader, valid_loader, criterion, optimizer, device, num_epochs, log_interval=1)

# Plot the training and validation AUC and loss
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(train_auc, label='Train AUC')
plt.plot(valid_auc, label='Validation AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()
plt.title('Training and Validation AUC')

plt.subplot(1, 2, 2)
plt.plot(train_loss, label='Train Loss')
plt.plot(valid_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.show()


## Model 4: MobilNet_v2

In [ ]:
# Load a pre-trained MobileNet model
model = models.mobilenet_v2(pretrained=True)
# Modify the input layer to accept a single channel (grayscale) image
model.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)

# Modify the final fully connected layer for binary classification
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Sequential(
    nn.Linear(num_ftrs, 128),
    nn.ReLU(),
    nn.Linear(128, num_classes),
    nn.Sigmoid()
)

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 50

In [ ]:
# Usage:
best_model, train_auc, valid_auc, train_loss, valid_loss = train_and_evaluate_model(
    model, train_loader, valid_loader, criterion, optimizer, device, num_epochs, log_interval=1)

# Plot the training and validation AUC and loss
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(train_auc, label='Train AUC')
plt.plot(valid_auc, label='Validation AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()
plt.title('Training and Validation AUC')

plt.subplot(1, 2, 2)
plt.plot(train_loss, label='Train Loss')
plt.plot(valid_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.show()
